<a href="https://colab.research.google.com/github/fegastal/PosTech-DataAnalytics_TechChallenge_3/blob/main/PNAD_Covid19_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install basedosdados

In [ ]:
import pandas as pd
import basedosdados as bd ## database management system for Bra# zilian public data
import numpy as np
import seaborn as sns
import os
from matplotlib import pyplot
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType
from pyspark.sql.functions import col, expr, concat, when, udf

spark = SparkSession.builder.appName("PNAD_Covic19").getOrCreate()
spark

In [ ]:
# dict_basedosdados = bd.read_table(dataset_id='br_ibge_pnad_covid',
# table_id='dicionario',
# billing_project_id="basedosdados-1")

In [ ]:
# base_df = bd.read_table(dataset_id='br_ibge_pnad_covid',
#   table_id='microdados',
#   billing_project_id="basedosdados-1") \
#           .fillna(0)

In [ ]:
# base_df_sql = bd.read_sql(query="SELECT  * \
#                                   FROM `basedosdados.br_ibge_pnad_covid.microdados` cov_md \
#                                   WHERE cov_md.mes in (9,10,11)") \
#                                 .fillna(0)


In [ ]:
# base_df_sql

In [ ]:
# Set display options to show all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

In [ ]:
# covid_df = spark.createDataFrame(data = base_df_sql)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df = spark.read.csv('/content/gdrive/MyDrive/colab_notebooks/pnad_covid19.csv',inferSchema = True,header=True)

AnalysisException: ignored

In [ ]:
type(df)

Selecting specific columns from data dictionary

In [ ]:
df = df.select(['UF','Ano','V1013','V1022','A002','A003','B0011','B0012','B0013','B0014','B0015','B0018',
                'B0019','B00111','B002','B005','B006','B007','C001','C007D','C007E','C01011','C013','D0051'
           ])

In [ ]:
df.show()

In [ ]:
# importing basedosdados dictionary for the table
df_dict = spark.createDataFrame(
    bd.read_table(dataset_id='br_ibge_pnad_covid',
    table_id='dicionario',
    billing_project_id="basedosdados-1").fillna(0)
)

In [ ]:
df_dict = df_dict.withColumn('chave',col('chave').cast('int')). \
                            fillna(0)

In [ ]:
df_dict.show()

In [ ]:
df_dict = df_dict.withColumn("chave_coluna", concat(col("nome_coluna"), col("chave")))

In [ ]:
df_dict.show()

In [ ]:
# for column_name in df.columns:
#     df = df.withColumnRenamed(column_name, column_name.lower())

In [ ]:
from pyspark.sql.functions import col
for column_name in df.columns:
  df = df.withColumn(column_name, col(column_name).cast('int'))

In [ ]:
df.show()

In [ ]:
df = df.fillna(0)
df.show()

## Altering data and column names

In [ ]:
df

In [ ]:
new_column_name_dict = {
    "UF": 'state_code',
    'Ano': 'year',
    'V1013': 'month',
    "V1022": "housing_status",
    "A002": "age",
    "A003": "sex",
    "B0011": "had_fever_last_week",
    "B0012": "had_cough_last_week",
    "B0013": "had_sore_throat_last_week",
    'B0014': "had_trouble_breathing_last_week",
    "B0015": "had_headache_last_week",
    "B0018": "had_runny_nose_last_week",
    "B0019": "had_lethargy_last_week",
    "B00111": "had_loss_of_smell_last_week",
    "B002": "went_to_healthcare_facility",
    "B005": "was_hospitalized",
    "B006": "used_ventilator",
    "B007": "has_health_insurance",
    "C001": "worked_last_week",
    "C007D": "main_job_activity",
    "C007E": "how_many_employees",
    "C01011": "income_range",
    "C013": "has_worked_remote",
    'D0051': "coronavirus_related_aid"
}

In [ ]:
for old_col, new_col in new_column_name_dict.items():
    df = df.withColumnRenamed(old_col, new_col)

In [ ]:
df.show()

In [ ]:
symptoms_dict = {
    1: "yes",
    2: "no",
    3: "dont_know",
    9: "ignored",
    0: "not_applicable"
}

In [ ]:
# Columns to translate
symptoms_cols = ['had_fever_last_week','had_cough_last_week',
                 'had_sore_throat_last_week','had_headache_last_week','had_runny_nose_last_week',
                 'had_lethargy_last_week','had_loss_of_smell_last_week','went_to_healthcare_facility','was_hospitalized','has_health_insurance',
                 'worked_last_week','has_worked_remote','coronavirus_related_aid','used_ventilator','had_trouble_breathing_last_week']

work_cols = ['main_job_activity','how_many_employees','income_range']

In [ ]:
brazil_states = {
    11: "RO",
    12: "AC",
    13: "AM",
    14: "RR",
    15: "PA",
    16: "AP",
    17: "TO",
    21: "MA",
    22: "PI",
    23: "CE",
    24: "RN",
    25: "PB",
    26: "PE",
    27: "AL",
    28: "SE",
    29: "BA",
    31: "MG",
    32: "ES",
    33: "RJ",
    35: "SP",
    41: "PR",
    42: "SC",
    43: "RS",
    50: "MS",
    51: "MT",
    52: "GO",
    53: "DF"
}

In [ ]:
sector_translation = {
    1: "agriculture_livestock_forestry_and_fishing",
    2: "extraction_of_petroleum_coal_metallic_minerals_stone_sand_salt_etc",
    3: "manufacturing_industry_including_clothing_and_homemade_manufacturing",
    4: "supply_of_electricity_and_gas_water_sewerage_and_garbage_collection",
    5: "construction",
    6: "wholesale_and_retail_trade",
    7: "repair_of_motor_vehicles_and_motorcycles",
    8: "passenger_transport",
    9: "freight_transport",
    10: "storage_post_and_delivery_services",
    11: "accommodation_hotels_inns_etc",
    12: "food_service_bars_restaurants_food_vendors",
    13: "information_and_communication_newspapers_radio_and_television_telecommunications_and_it",
    14: "banks_financial_and_insurance_activities",
    15: "real_estate_activities",
    16: "law_firms_engineering_advertising_and_veterinary_services_professional_scientific_and_technical_activities",
    17: "labor_leasing_security_cleaning_landscaping_and_call_center_activities",
    18: "public_administration_federal_state_and_municipal_government",
    19: "education",
    20: "human_health_and_social_assistance",
    21: "religious_organizations_unions_and_associations",
    22: "artistic_sports_and_recreational_activities",
    23: "hairdressing_beauty_treatment_and_personal_services",
    24: "paid_domestic_service_to_be_reported_under_occupation",
    25: "other",
    0: "not_applicable"
}

In [ ]:
employee_translation = {
    1: "1_to_5",
    2: "6_to_10",
    3: "11+",
    0: "not_applicable"
}

In [ ]:
income_range = {
    0: "0-100",
    1: "101-300",
    2: "301-600",
    3: "601-800",
    4: "801-1,600",
    5: "1,601-3,000",
    6: "3,001-10,000",
    7: "10,001-50,000",
    8: "50,001-100,000",
    9: "100,000+",
    0: "not_applicable"
}

In [ ]:
def translate_and_cast(value):
    return symptoms_dict.get(int(value), None)

translate_udf = udf(translate_and_cast, StringType())

# Apply the UDF to the selected columns
for column in symptoms_cols:
    df = df.withColumn(column, translate_udf(col(column)))

In [ ]:
df.schema

In [ ]:
df.show()

In [ ]:
def translate_and_cast_state(value):
    return brazil_states.get(value, None)

translate_udf = udf(translate_and_cast_state, StringType())

# Apply the UDF to the selected columns
df = df.withColumn("state_code", col("state_code").cast(IntegerType()))
df = df.withColumn('state_code', translate_udf(col('state_code')))

In [ ]:
df.show()

In [ ]:
def translate_and_cast_sector(value):
    return sector_translation.get(value, None)

translate_udf = udf(translate_and_cast_sector, StringType())

# Apply the UDF to the selected columns

df = df.withColumn('main_job_activity', translate_udf(col('main_job_activity')))

In [ ]:
df.show()

In [ ]:
def translate_and_cast_n_of_employees(value):
    return employee_translation.get(value, None)

translate_udf = udf(translate_and_cast_sector, StringType())

# Apply the UDF to the selected columns

df = df.withColumn('how_many_employees', translate_udf(col('how_many_employees')))

In [ ]:
df.show()

In [ ]:
def translate_and_cast_income(value):
    return income_range.get(value, None)

translate_udf = udf(translate_and_cast_income, StringType())

# Apply the UDF to the selected columns

df = df.withColumn('income_range', translate_udf(col('income_range')))

In [ ]:
df = df.withColumn("sex", F.expr("CASE WHEN sex == 1 THEN 'male' ELSE 'female' END"))
df = df.withColumn("housing_status", F.expr("CASE WHEN housing_status == 1 THEN 'urban' ELSE 'rural' END"))

In [ ]:
df.show()

In [ ]:
df.count()

In [ ]:
# creating has_covid column based on the symptoms column

symptom_col = ['had_fever_last_week','had_cough_last_week',
                 'had_sore_throat_last_week','had_headache_last_week','had_runny_nose_last_week',
                 'had_lethargy_last_week','had_loss_of_smell_last_week']

## if a row has a count of symptoms equal or greater than 2, we identify this research as positive for Covid
yes_count = sum(when(col(c) == "yes", 1).otherwise(0) for c in symptom_col)

In [ ]:
# Use withColumn to create the has_covid column
df = df.withColumn("has_covid", yes_count >= 2)


In [ ]:
df.filter(df.has_covid==True).show()

## Ploting results

In [ ]:
df.show()

In [ ]:
df_pd_state_covid = df.select(['state_code','has_covid','was_hospitalized','has_health_insurance','used_ventilator','went_to_healthcare_facility']).toPandas()

In [ ]:
df_pd_state_covid

## Covid cases per state

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
filtered_df = df_pd_state_covid.query('has_covid==True')
filtered_df

In [ ]:
# setting plot size
plt.figure(figsize=(10, 6))

# creating sns barchart
ax = sns.barplot(x=filtered_df['state_code'].value_counts().index,
                 y=filtered_df['state_code'].value_counts().values,
                 palette='rocket')

# Title and label
plt.xlabel('State Code')
plt.ylabel('Count')
plt.title('COVID-19 Cases by State, PNAD Contínua Sep/20 to Nov/20')

#adding data label
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.0f'),
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha = 'center', va = 'center',
                size=10,
                xytext = (0, 9),
                textcoords = 'offset points')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# setting plot size
plt.figure(figsize=(15, 6))

# creating sns barchart
ax = sns.countplot(data=filtered_df.query('was_hospitalized == "yes" | was_hospitalized=="no"'),
                 x= 'state_code',
                 hue='was_hospitalized',
                 palette='rocket')

# Title and label
plt.xlabel('State Code')
plt.ylabel('Count')
plt.title('Hospitalized COVID-19 Cases by State, PNAD Contínua Sep/20 to Nov/20')

#adding data label
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.0f'),
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha = 'center', va = 'center',
                size=10,
                xytext = (0, 9),
                textcoords = 'offset points')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show

In [ ]:
# setting plot size
plt.figure(figsize=(15, 6))

# creating sns barchart
ax = sns.countplot(data=filtered_df.query('was_hospitalized== "yes" & (used_ventilator == "yes" | used_ventilator=="no")'),
                 x= 'state_code',
                 hue='used_ventilator',
                 palette='rocket')

# Title and label
plt.xlabel('State Code')
plt.ylabel('Count')
plt.title('Subject used ventilator Cases by State, PNAD Contínua Sep/20 to Nov/20')

#adding data label
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.0f'),
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha = 'center', va = 'center',
                size=10,
                xytext = (0, 9),
                textcoords = 'offset points')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show

In [ ]:
# @title
# setting plot size
plt.figure(figsize=(15, 6))

# creating sns barchart
ax = sns.countplot(data=filtered_df.query('went_to_healthcare_facility == "yes" & has_health_insurance != "not_applicable"'),
                 x= 'state_code',
                 hue='has_health_insurance',
                 orient = 'h',
                 palette='rocket')

# Title and label
plt.xlabel('State Code')
plt.ylabel('Count')
plt.title('Subject sought healthcare facility, has_health_insurance breakdown, PNAD Contínua Sep/20 to Nov/20')

#adding data label
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.0f'),
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha = 'center', va = 'center', rotation = 45,
                size=8,
                xytext = (0, 9),
                textcoords = 'offset points')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show

## Connecting PySpark to BigQuery

In [ ]:
from google.cloud import bigquery
import pandas_gbq

In [ ]:
output_path = "/content/gdrive/MyDrive/colab_notebooks/pnad_covid19_treated.csv"

# Use the write method to save the DataFrame as a CSV file
df.write.csv(output_path, header=True, mode="overwrite")

In [ ]:
# Use the Cloud Storage bucket for temporary BigQuery export data used
# by the connector.
bucket = "pnad_covid19"
spark.conf.set('temporaryGcsBucket', bucket)

In [ ]:
# df.write.format('bigquery') \
#   .option('table', 'pnad_covid29.covid19') \
#   .save()